In [1]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import napari
import numpy as np
from mpl_interactions import hyperslicer
from pymmcore_mda_engines import DriftCorrectionEngine, ImageGenerator
from pymmcore_plus import CMMCorePlus
from skimage.registration import phase_cross_correlation
from useq import MDASequence

In [2]:
v = napari.Viewer()
dw, main_window = v.window.add_plugin_dock_widget("napari-micromanager")
main_window.mda.acquisition_order_comboBox.setCurrentText("tpcz")
# main_window.mda
core = CMMCorePlus.instance()
core.loadSystemConfiguration()
img_gen = ImageGenerator(5000, radius_loc=15, step_scale=(1,1), radius_scale=3, XY_stage_drift=(10, 2))
engine = DriftCorrectionEngine()
core.register_mda_engine(engine)
engine.register_image_generator(img_gen)

mda = MDASequence(
    # stage_positions=[(0, 100, 30), (700, 150, 35)],
    stage_positions=[(0, 100, 30)],
    channels=["DAPI"],
    time_plan={"interval": 0.25, "loops": 25},
    # z_plan={"range": 4, "step": 0.5},
    axis_order="tpcz",
)


2022-05-20 11:27:28.436 | DEBUG    | pymmcore_plus._util:find_micromanager:47 - using MM path found in applications: /usr/local/lib/micro-manager
2022-05-20 11:27:28.438 | INFO     | pymmcore_plus.core._mmcore_plus:setDeviceAdapterSearchPaths:152 - setting adapter search paths: ['/usr/local/lib/micro-manager']
2022-05-20 11:27:28.699 | DEBUG    | pymmcore_plus._util:find_micromanager:47 - using MM path found in applications: /usr/local/lib/micro-manager


## Make a minimal algorithim to correct for drift

Using a simple comparison of pixel shift relative to the previous frame in the same position. While we kept this simple for the demo there are lots of ways we could make this more sophisticated. Crucially expanding on this doesn't require starting from scratch, we can easily build on this.

In [3]:
from collections import defaultdict


class DriftAdjuster:
    def __init__(self, engine: DriftCorrectionEngine):
        self._engine = engine
        self._lastImage = {}
        self._engine.events.frameReady.connect(self._onFrame)
        self._engine.events.sequenceStarted.connect(self._onStart)
        self._active=True

    def _onStart(self, sequence: MDASequence):
        # clear our cache of images
        self._lastImage = {}

    def _onFrame(self, img, event):
        
        p = event.index["p"]
        # grab our stored last image in this position
        prev_image = self._lastImage.get(p, None)
    
        if prev_image is not None and self._active:
            # calculate the drift correction amount
            # this could get much more sophisticated!
            adjustment = phase_cross_correlation(prev_image, img)
            self._engine.drift_correction[p] += adjustment[0][:2]

        self._lastImage[event.index["p"]] = img

    # below makes life nicer but is not critical
    @property
    def active(self) -> bool:
        return self._active

    @active.setter
    def active(self, val: bool):
        self._active = bool(val)




In [4]:
adjuster = DriftAdjuster(engine)
# adjuster.active = False

2022-05-20 11:26:23.454 | INFO     | pymmcore_plus.mda._engine:_prepare_to_run:122 - MDA Started: Multi-Dimensional Acquisition ▶ nt: 20, np: 2, nc: 1, nz: 0
2022-05-20 11:26:23.459 | INFO     | pymmcore_mda_engines._engines:run:67 - metadata={} index={'t': 0, 'p': 0, 'c': 0} channel=Channel(config='Cy5') exposure=100.0 min_start_time=0.0 x_pos=-0.0 y_pos=-0.0 z_pos=None properties=None
2022-05-20 11:26:23.590 | INFO     | pymmcore_mda_engines._engines:run:67 - metadata={} index={'t': 0, 'p': 1, 'c': 0} channel=Channel(config='Cy5') exposure=100.0 min_start_time=0.0 x_pos=1999.995 y_pos=-0.0 z_pos=None properties=None
2022-05-20 11:26:23.835 | INFO     | pymmcore_mda_engines._engines:run:67 - metadata={} index={'t': 1, 'p': 0, 'c': 0} channel=Channel(config='Cy5') exposure=100.0 min_start_time=0.001 x_pos=-0.0 y_pos=-0.0 z_pos=None properties=None
2022-05-20 11:26:23.916 | INFO     | pymmcore_mda_engines._engines:run:67 - metadata={} index={'t': 1, 'p': 1, 'c': 0} channel=Channel(confi

In [5]:
adjuster.active = False

2022-05-20 11:23:52.473 | INFO     | pymmcore_plus.mda._engine:_prepare_to_run:122 - MDA Started: Multi-Dimensional Acquisition ▶ nt: 20, np: 2, nc: 1, nz: 0
2022-05-20 11:23:52.496 | INFO     | pymmcore_mda_engines._engines:run:67 - metadata={} index={'t': 0, 'p': 0, 'c': 0} channel=Channel(config='Cy5') exposure=100.0 min_start_time=0.0 x_pos=-0.0 y_pos=-0.0 z_pos=None properties=None
2022-05-20 11:23:52.829 | INFO     | pymmcore_mda_engines._engines:run:67 - metadata={} index={'t': 0, 'p': 1, 'c': 0} channel=Channel(config='Cy5') exposure=100.0 min_start_time=0.0 x_pos=-0.0 y_pos=1999.995 z_pos=None properties=None
2022-05-20 11:23:53.495 | INFO     | pymmcore_mda_engines._engines:run:67 - metadata={} index={'t': 1, 'p': 0, 'c': 0} channel=Channel(config='Cy5') exposure=100.0 min_start_time=1.0 x_pos=-0.0 y_pos=-0.0 z_pos=None properties=None
2022-05-20 11:23:53.745 | INFO     | pymmcore_mda_engines._engines:run:67 - metadata={} index={'t': 1, 'p': 1, 'c': 0} channel=Channel(config=